In [1]:
import sys
sys.path.insert(0,'/home/csunix/schtmt/NewFolder/caffe_Sep_anaconda/python')
import numpy as np
import matplotlib.pyplot as plt
import caffe
import h5py
import cv2
import os
import time
caffe.set_device(0)
caffe.set_mode_gpu()

# aug1
net = caffe.Net('test2.prototxt',
                '../convLSTM_models/ped1Recons_aug1.caffemodel', 
                caffe.TEST) 

In [3]:
import fnmatch
import os
from PIL import Image
# Ped1
frame_path='/usr/not-backed-up/1_DATABASE/UCSD_Anomaly_Dataset.tar/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/'

save_path = '/usr/not-backed-up/1_convlstm/Ped1_AE6/'
if not os.path.exists(save_path):
    os.mkdir(save_path)
total_time = 0
total_frame = 0
for seq in range(36): 
    print(seq)    
    # load images
    if seq < 9:                
        dirpath = (frame_path + 'Test00%d' % (seq+1))
    else:
        dirpath = (frame_path + 'Test0%d' % (seq+1))
    nfr = len(fnmatch.filter(os.listdir(dirpath), '*.tif'))
#         frame_seq = np.zeros((227,227,nfr),dtype=np.float64)
    for fr in range(nfr):            
        if fr < 9:
            fr_id = '00%d' % (fr+1)                                       
        elif fr < 99:
            fr_id = '0%d' % (fr+1)                
        else:
            fr_id = '%d' % (fr+1)                
        im1 = Image.open(dirpath + '/' + fr_id + '.tif')
        im1 = np.array(im1.resize((227,227),Image.BILINEAR))#size [width,height]            
        im1 = np.reshape(im1,[1,227,227])
        im1 = im1.astype(float) / 255.
        if fr == 0:
            frame_seq = im1
        else:
            frame_seq = np.append(frame_seq,im1,0)        
    start_time = time.clock()
    frame_error = np.zeros([1,nfr-4])
    for fr in range(nfr-4): 
        cur_vol = frame_seq[fr:fr+5,:,:]
        gt = np.zeros(cur_vol.shape)
        for ix in range(5):
            gt[ix,:,:] = cur_vol[4-ix,:,:]        
        
        net.blobs['input'].data[...] = cur_vol.reshape([1,5,1,227,227])
        out = net.forward()        
        recons = net.blobs['deconv3'].data
        recons = np.reshape(recons,[5,227,227])
        err = np.power((gt - recons),2)
#         # error of 5 frames
        err = np.reshape(err,[5,227*227])
        err = np.sqrt(np.sum(err,1))
        frame_error[:,fr] = np.sum(err)  
    total_time = total_time + time.clock() - start_time
    total_frame = total_frame + nfr
    filename = ("test_%d_error.h5" % (seq+1))        

    h5f = h5py.File(os.path.join(save_path,filename), 'w')
    h5f.create_dataset('frame_error', data=frame_error)                
    h5f.close()     
     

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
